In [16]:
# !conda install -c conda-forge librosa
# !conda install tqdm

import torch as ch
import torch.nn as nn
import torch.functional as F
import torch.utils.data as Data

import numpy as np
import pandas as pd
import scipy as sp
from matplotlib import pyplot as plt
%matplotlib inline

import os
import string
import librosa
import tqdm
import shutil
# !pip install lws
import lws
import IPython.display as ipd
from importlib import reload

import hyperparams
reload(hyperparams)
params = hyperparams.Hyperparams
import datasets
reload(datasets)
import dcttsModel
reload(dcttsModel)
import utils
reload(utils)

<module 'utils' from '/home/aduriseti/tts/ettts/utils.py'>

In [17]:
# # textEnc = TextEnc(d=d,e=e,c2i=c2i)
# # audioEnc = AudioEnc(d=d,F=F)
# audioDec = AudioDec(d=params.d,F=params.F).to(dev)

ssrn = dcttsModel.SSRN(c=params.c,F=params.F,Fp=params.Fp)
text2Mel = dcttsModel.Text2Mel(d=params.d,e=params.e,c2i=params.c2i,F=params.F)

# print tunable hyperparams
print('HYPERPARAMS',utils.paramDict)
[sum(np.prod(p.size()) for p in m.parameters()) for m in (text2Mel,ssrn)]

HYPERPARAMS {'sep': 3, 'alpha': 1, 'dropout': 0.05, 'norm': 2, 'lr': 0.001, 'chunk': 1}


[1601042, 2611841]

In [18]:
ssrn

SSRN(
  (seq): Sequential(
    (0): C(
      (reduce): Conv1d(80, 5, kernel_size=(1,), stride=(1,))
      (conv): Conv1d(5, 32, kernel_size=(1,), stride=(1,))
      (expand): Conv1d(32, 512, kernel_size=(1,), stride=(1,))
      (norm): LayerNorm(torch.Size([512]), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.05)
    )
    (1): HC(
      (conv): C(
        (reduce): Conv1d(512, 32, kernel_size=(1,), stride=(1,))
        (conv): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (expand): Conv1d(64, 1024, kernel_size=(1,), stride=(1,))
        (norm): LayerNorm(torch.Size([1024]), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.05)
      )
    )
    (2): HC(
      (conv): C(
        (reduce): Conv1d(512, 32, kernel_size=(1,), stride=(1,))
        (conv): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
        (expand): Conv1d(64, 1024, kernel_size=(1,), stride=(1,))
        (norm): LayerNorm(torch.Size([10

In [19]:
L,S,Y = datasets.LJSpeechDataset(ttmel=1).__getitem__(100)
L,S,Y = L.unsqueeze(0),S.unsqueeze(0),Y.unsqueeze(0)

# initial audio enc
# S = ch.zeros((80,217)).to(dev).unsqueeze(0)
s0 = ch.zeros((80,params.chunk)).unsqueeze(0)
Shat = s0

In [20]:
for dev in ('cpu','cuda:0') if ch.cuda.is_available() else ('cpu'):
    print('DEVICE',dev)
    L,S,Y = L.to(dev),S.to(dev),Y.to(dev)
    s0 = ch.zeros((80,params.chunk)).unsqueeze(0).to(dev)
    Shat = s0
    text2Mel = text2Mel.eval().to(dev)
    ssrn = ssrn.eval().to(dev)
    with ch.no_grad():
        for t in tqdm.tqdm_notebook(range(218//params.chunk)):
            Snext,A = text2Mel(L,Shat[:,:,:])
    #         S[:,:,t+1 ] = Shat[:,:,t]
            Shat = ch.cat((s0,Snext),dim=2)
    #         Shat = ch.cat((Shat,Snext[:,:,-params.chunk:]),dim=2)

DEVICE cpu



DEVICE cuda:0


In [15]:
Shat.shape

torch.Size([1, 80, 2])

In [ ]:
Yhat = ssrn(Shat)